Step 1: Prepare Data in PySpark

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkSQLExercise") \
    .getOrCreate()

In [5]:
from pyspark.sql import Row
data = [
Row(EmpID=101, Name="Ravi", Department="Engineering", Project="AI Engine",
Salary=95000, HoursPerWeek=42),
Row(EmpID=102, Name="Sneha", Department="Engineering", Project="Data Platform",
Salary=87000, HoursPerWeek=45),
Row(EmpID=103, Name="Kabir", Department="Marketing", Project="Product Launch",
Salary=65000, HoursPerWeek=40),
Row(EmpID=104, Name="Anita", Department="Sales", Project="Client Outreach",
Salary=70000, HoursPerWeek=38),
Row(EmpID=105, Name="Divya", Department="Engineering", Project="AI Engine",
Salary=99000, HoursPerWeek=48),
Row(EmpID=106, Name="Amit", Department="Marketing", Project="Social Media",
Salary=62000, HoursPerWeek=35),
Row(EmpID=107, Name="Priya", Department="HR", Project="Policy Revamp",
Salary=58000, HoursPerWeek=37),
Row(EmpID=108, Name="Manav", Department="Sales", Project="Lead Gen", Salary=73000,
HoursPerWeek=41),
Row(EmpID=109, Name="Neha", Department="Engineering", Project="Security Suite",
Salary=91000, HoursPerWeek=46),
Row(EmpID=110, Name="Farah", Department="HR", Project="Onboarding", Salary=60000,
HoursPerWeek=36)
]
df = spark.createDataFrame(data)
df.show(truncate=False)

+-----+-----+-----------+---------------+------+------------+
|EmpID|Name |Department |Project        |Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|101  |Ravi |Engineering|AI Engine      |95000 |42          |
|102  |Sneha|Engineering|Data Platform  |87000 |45          |
|103  |Kabir|Marketing  |Product Launch |65000 |40          |
|104  |Anita|Sales      |Client Outreach|70000 |38          |
|105  |Divya|Engineering|AI Engine      |99000 |48          |
|106  |Amit |Marketing  |Social Media   |62000 |35          |
|107  |Priya|HR         |Policy Revamp  |58000 |37          |
|108  |Manav|Sales      |Lead Gen       |73000 |41          |
|109  |Neha |Engineering|Security Suite |91000 |46          |
|110  |Farah|HR         |Onboarding     |60000 |36          |
+-----+-----+-----------+---------------+------+------------+



In [7]:
df.createOrReplaceTempView("employees_local")
df.createOrReplaceGlobalTempView("employees_global")

Part A: Exercises on Local View ( employees_local )

In [12]:
spark.sql("select * from employees_local where Project='AI Engine'").show()

+-----+-----+-----------+---------+------+------------+
|EmpID| Name| Department|  Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------+------+------------+
|  101| Ravi|Engineering|AI Engine| 95000|          42|
|  105|Divya|Engineering|AI Engine| 99000|          48|
+-----+-----+-----------+---------+------+------------+



In [15]:
spark.sql("select * from employees_local where Department='Marketing' and Salary>60000").show()

+-----+-----+----------+--------------+------+------------+
|EmpID| Name|Department|       Project|Salary|HoursPerWeek|
+-----+-----+----------+--------------+------+------------+
|  103|Kabir| Marketing|Product Launch| 65000|          40|
|  106| Amit| Marketing|  Social Media| 62000|          35|
+-----+-----+----------+--------------+------+------------+



In [19]:
spark.sql("select Department, avg(Salary) as avg_salary, Department from employees_local group by Department ").show()

+-----------+----------+-----------+
| Department|avg_salary| Department|
+-----------+----------+-----------+
|      Sales|   71500.0|      Sales|
|Engineering|   93000.0|Engineering|
|  Marketing|   63500.0|  Marketing|
|         HR|   59000.0|         HR|
+-----------+----------+-----------+



In [21]:
spark.sql("select Name, Salary from employees_local order by Salary desc limit 3").show()

+-----+------+
| Name|Salary|
+-----+------+
|Divya| 99000|
| Ravi| 95000|
| Neha| 91000|
+-----+------+



In [22]:
spark.sql("Select * from employees_local where HoursPerWeek >40").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  108|Manav|      Sales|      Lead Gen| 73000|          41|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



In [25]:
spark.sql("select Project, count(Project) as tot_count from employees_local group by Project ").show()

+---------------+---------+
|        Project|tot_count|
+---------------+---------+
|  Data Platform|        1|
|      AI Engine|        2|
| Product Launch|        1|
|Client Outreach|        1|
| Security Suite|        1|
|  Policy Revamp|        1|
|       Lead Gen|        1|
|   Social Media|        1|
|     Onboarding|        1|
+---------------+---------+



In [26]:
spark.catalog.dropTempView("employees_local")


True

In [ ]:
spark.sql("select * from employees_local").show()


Part B: Exercises on Global View ( employees_global )

In [28]:
spark.sql("select * from global_temp.employees_global where Department='HR' and HoursPerWeek < 38").show()

+-----+-----+----------+-------------+------+------------+
|EmpID| Name|Department|      Project|Salary|HoursPerWeek|
+-----+-----+----------+-------------+------+------------+
|  107|Priya|        HR|Policy Revamp| 58000|          37|
|  110|Farah|        HR|   Onboarding| 60000|          36|
+-----+-----+----------+-------------+------+------------+



In [29]:
spark.sql("select Department, sum(Salary) as total_salary_payout from global_temp.employees_global group by Department").show()

+-----------+-------------------+
| Department|total_salary_payout|
+-----------+-------------------+
|      Sales|             143000|
|Engineering|             372000|
|  Marketing|             127000|
|         HR|             118000|
+-----------+-------------------+



In [33]:
from pyspark.sql.functions import when, col
df_status=df.withColumn("Status",when(col("HoursPerWeek")>45, "OverWorked").otherwise("Normal")).show()

+-----+-----+-----------+---------------+------+------------+----------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|    Status|
+-----+-----+-----------+---------------+------+------------+----------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|    Normal|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|    Normal|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|    Normal|
|  104|Anita|      Sales|Client Outreach| 70000|          38|    Normal|
|  105|Divya|Engineering|      AI Engine| 99000|          48|OverWorked|
|  106| Amit|  Marketing|   Social Media| 62000|          35|    Normal|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|    Normal|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|    Normal|
|  109| Neha|Engineering| Security Suite| 91000|          46|OverWorked|
|  110|Farah|         HR|     Onboarding| 60000|          36|    Normal|
+-----+-----+-----------+---------------+------+---

In [36]:
spark.sql("select Project, count(Project) as Tot_person from global_temp.employees_global group by Project").show()

+---------------+----------+
|        Project|Tot_person|
+---------------+----------+
|  Data Platform|         1|
|      AI Engine|         2|
| Product Launch|         1|
|Client Outreach|         1|
| Security Suite|         1|
|  Policy Revamp|         1|
|       Lead Gen|         1|
|   Social Media|         1|
|     Onboarding|         1|
+---------------+----------+



In [42]:
spark.sql("""
select e.EmpId, e.Name, e.Salary
from global_temp.employees_global e
join(
  select Department,avg(Salary) as avg_salary
  from global_temp.employees_global
  group by Department
) d_avg
 on e.Department = d_avg.Department
 where e.salary > d_avg.avg_salary
 """).show()


+-----+-----+------+
|EmpId| Name|Salary|
+-----+-----+------+
|  101| Ravi| 95000|
|  105|Divya| 99000|
|  103|Kabir| 65000|
|  108|Manav| 73000|
|  110|Farah| 60000|
+-----+-----+------+



In [43]:
from pyspark.sql import SparkSession
new_spark = SparkSession.builder \
    .appName("SparkSQLExercise") \
    .getOrCreate()

In [ ]:
spark_new.sql("selct * from global_temp.employees_global").show()


Bonus Challenges

In [46]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col
window_spec = Window.partitionBy("Department").orderBy(col("Salary").desc())
df_ranked = df.withColumn("salary_rank", rank().over(window_spec))
df_ranked.show()


+-----+-----+-----------+---------------+------+------------+-----------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|salary_rank|
+-----+-----+-----------+---------------+------+------------+-----------+
|  105|Divya|Engineering|      AI Engine| 99000|          48|          1|
|  101| Ravi|Engineering|      AI Engine| 95000|          42|          2|
|  109| Neha|Engineering| Security Suite| 91000|          46|          3|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|          4|
|  110|Farah|         HR|     Onboarding| 60000|          36|          1|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|          2|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|          1|
|  106| Amit|  Marketing|   Social Media| 62000|          35|          2|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|          1|
|  104|Anita|      Sales|Client Outreach| 70000|          38|          2|
+-----+-----+-----------+-------------

In [49]:
df_engineering = df_ranked.filter(col("Department") == "Engineering")
df_engineering.createOrReplaceTempView("engineering_employees")


In [59]:
df.createOrReplaceTempView("employees_local")
spark.sql("""
Select * from employees_local
where HoursPerWeek >= 38
""").show()


+-----+-----+-----------+---------------+------+------------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|
|  104|Anita|      Sales|Client Outreach| 70000|          38|
|  105|Divya|Engineering|      AI Engine| 99000|          48|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|
|  109| Neha|Engineering| Security Suite| 91000|          46|
+-----+-----+-----------+---------------+------+------------+

